In [1]:
#day22
#
#https://adventofcode.com/2021/day/22

import functools

def getRange(x):
    v = x.split("..")
    return int(v[0]), int(v[1])

def getAxis(x):
    v = x.split("=")
    return getRange(v[1])

def parse(text):
    #print(text)
    inst=[]
    lines=text.split("\n")
    for l in lines:
        state, rng= l.split(" ")
        #print(state, rng)
        axes=rng.split(",")
        #print(axes)
        vals = [getAxis(a) for a in axes]
        
        inst.append((state,vals))
    return inst

############################
#
#  Axis
#
############################

#Model sampling as falling panes of glass
#use fact that axes can be treated seperately
#Axes can be combined by knowing if we hit in all axis hitters remain 
#else they miss

class Axis:
    #ranges are natural numbers
    #end values are inclusive
    def __init__(s, r):
        s.r = r
        assert(isinstance(r, tuple))
        assert(len(list(r)) == 2)
        assert(isinstance(r[0], int))
        assert(isinstance(r[1], int))
        assert(r[0]<=r[1])
    
    def __str__(s):
        return "%dto%d"%(s.r[0], s.r[1])
    
    def draw(s, c, w):
        l = " "*(s.r[0])
        l+= c * (s.r[1]-s.r[0] + 1)
        l+= " " * (w-len(l))
        l+= str(s.r)
        print(l)
        
    def l(s):
        return s.r[1]-s.r[0]+1
        
    def shatter_against(s, other):
        h0,h1 = s.r
        s0,s1 = other.r
        p = []
        
        #low miss part
        if h0<s0:
            p.append((Axis((h0,min(h1,s0-1))),"m"))
        
        #middle hit part
        if h1>=s0 and h0<=s1:
            p.append((Axis((max(h0,s0),min(h1,s1))),"h"))
         
        #high miss part
        if s1<h1:
            p.append((Axis((max(s1+1,h0),h1)),"m"))
            
        return p
    
    def covers(s, other):
        return s.r[0] <= other.r[0] and s.r[1] >= other.r[1]
    
def ran(x0, l):
    return (x0, x0+l-1)

def test_axis(len_hitter, len_stator, margin):
    W=20
    field_length = (len_hitter + margin)*2 + len_stator
    print("hitter:%d stator:%d margin:%d"%(len_hitter, len_stator, margin))
    for i in range(field_length - len_hitter + 1):
        stator = Axis(ran(len_hitter+margin, len_stator))
        hitter = Axis(ran(i,len_hitter))
        pieces = hitter.shatter_against(stator)
        hitter.draw("H",W)
        stator.draw("S",W)
        if len(pieces) == 0:
            print("!"*W)
        for i in range(len(pieces)):
            pieces[i][0].draw(pieces[i][1],W)
        print("-"*W)
            
#test_axis(2,4,1)
#test_axis(3,3,1)
#test_axis(4,2,1)

############################
#
#  Algorithm
#
############################


#with a robust axis the task can be solved
#by making a set of three axis covering the
#scan area and shatter each axis in the 
#operations in reverse order. Volumes that 
#hit on all axis adds to the accumulators and missing volumes
#continue through remaining operations. At the end 
#remaining are counted as off.
#
#A note on performance. The most significant effect on performance 
#was merging shattered areas. It vastly reduced the hitter list size 
#and kept runtime fast.
#
#It was also tried to filter out operations if they were "occluded"
#by later operations. But it hat little effect

def merge(a,b):
    #return None
    ax,ay,az = a
    bx,by,bz = b
    if ay == by and az == bz:
        if ax[1]+1==bx[0]:
            return ((ax[0], bx[1]),ay,az)    
        if bx[1]+1==ax[0]:
            return ((bx[0], ax[1]),ay,az)    
    if ax == bx and az == bz:
        if ay[1]+1==by[0]:
            return (ax,(ay[0], by[1]),az)    
        if by[1]+1==ay[0]:
            return (ax,(by[0], ay[1]),az)    
    if ax == bx and ay == by:
        if az[1]+1==bz[0]:
            return (ax,ay,(az[0], bz[1]))    
        if by[1]+1==ay[0]:
            return (ax,ay,(bz[0], az[1]))    
    return None   
        

def solve1(fn, part, facit):
    print("Part %d on %s"%(part, fn))
    text = open(fn).read()
    r=50
    instructions = parse(text)
    if part == 2:
        for t in instructions:
            inst, ranges = t
            for a,b in ranges:
                r = max(r,max(abs(a),abs(b)))
    
    scanarea=((-r,r),(-r,r),(-r,r))
    hitters=[scanarea]
    scanvolume= Axis(scanarea[0]).l()*Axis(scanarea[1]).l()*Axis(scanarea[2]).l()
    off=0
    on=0
    instructions = list(reversed(instructions))
    instlen = len(instructions)
    
    while len(instructions) > 0:
        t = instructions.pop(0)
        inst, ranges = t
        rx,ry,rz=ranges
        
        print("%d/%d"%(len(instructions),instlen), inst, rx, ry, rz, "hitlist:", len(hitters))
        x=Axis(rx)
        y=Axis(ry)
        z=Axis(rz)
            
        next_hitters=[]
        for hx,hy,hz in hitters:
            ax = Axis(hx).shatter_against(x)
            ay = Axis(hy).shatter_against(y)
            az = Axis(hz).shatter_against(z)
            tmp = []
            for xx,tx in ax:
                for yy,ty in ay:
                    for zz,tz in az:
                        vol = xx.l()*yy.l()*zz.l()
                        #print(tx, ty, tz)
                        if tx==ty==tz=="h":
                            if inst=="on":
                                on+=vol
                            else:
                                off+=vol
                        else:
                            tmp.append((xx.r,yy.r,zz.r))    
            
            #Newly generated volumes are highly 
            #mergeable
            if len(tmp) > 1:
                for i in range(len(tmp)):
                    a = tmp.pop(0)
                    for j in range(len(tmp)):
                        b = tmp.pop(0)
                        m = merge(a,b)
                        if not m is None:
                            a = m
                            break 
                        tmp.append(b)
                    tmp.append(a)
            next_hitters += tmp
            
        hitters=next_hitters
        
    for hx,hy,hz in hitters:
        vol += Axis(hx).l()*Axis(hy).l()*Axis(hz).l()
        off += vol
    print("fn:", fn)
    print("part:", part)
    print("on:", on)
    print("off:", off)
    print("scanvolume:", scanvolume)
    if not facit is None:
        if on != facit:
            print("on were supposed to be", facit)
            assert(0)
        else:
            print("Correct!!!")
            print("="*50)
            
            #assert(0)
    
solve1("i22_test.txt",1,590784)    
solve1("i22.txt",1, 650099)

solve1("i22_test2.txt",2,2758514936282235)    
solve1("i22.txt",2,1254011191104293)


Part 1 on i22_test.txt
21/22 on (967, 23432) (45373, 81175) (27513, 53682) hitlist: 1
20/22 on (-54112, -39298) (-85059, -49293) (-27449, 7877) hitlist: 1
19/22 on (-41, 9) (-7, 43) (-33, 15) hitlist: 1
18/22 off (18, 30) (-20, -8) (-3, 13) hitlist: 9
17/22 on (-49, -5) (-3, 45) (-29, 18) hitlist: 21
16/22 off (-32, -23) (11, 30) (-14, 3) hitlist: 34
15/22 on (-16, 35) (-41, 10) (-47, 6) hitlist: 50
14/22 off (-40, -22) (-38, -28) (23, 41) hitlist: 69
13/22 on (-18, 26) (-33, 15) (-7, 46) hitlist: 97
12/22 off (-48, -32) (-32, -16) (-15, -5) hitlist: 127
11/22 on (-12, 35) (6, 50) (-50, -2) hitlist: 152
10/22 off (-48, -32) (26, 41) (-47, -37) hitlist: 152
9/22 on (-22, 26) (-27, 20) (-29, 19) hitlist: 174
8/22 on (-30, 21) (-8, 43) (-13, 34) hitlist: 188
7/22 on (-39, 5) (-6, 47) (-3, 44) hitlist: 205
6/22 on (-27, 23) (-28, 26) (-21, 29) hitlist: 221
5/22 on (2, 47) (-22, 22) (-23, 27) hitlist: 241
4/22 on (-49, 1) (-3, 46) (-24, 28) hitlist: 256
3/22 on (-46, 7) (-6, 46) (-50, -1) h

303/420 on (-27137, -3917) (-82970, -73972) (-7640, 2780) hitlist: 1
302/420 on (45535, 66832) (30184, 50465) (40383, 65657) hitlist: 1
301/420 on (-12144, -4590) (-86532, -70213) (-27908, -18632) hitlist: 1
300/420 off (42008, 72354) (29685, 47325) (28537, 41099) hitlist: 1
299/420 on (53088, 71926) (33364, 61477) (-32978, -3454) hitlist: 1
298/420 off (-61367, -50375) (-46813, -36824) (26048, 47897) hitlist: 1
297/420 on (29370, 47558) (-89453, -60476) (-7077, 24097) hitlist: 1
296/420 off (9879, 18027) (-88040, -55114) (-31569, -21687) hitlist: 1
295/420 off (-41292, -28792) (-82764, -50053) (-18368, 1547) hitlist: 1
294/420 off (-47333, -40889) (-23607, -16074) (-75438, -61498) hitlist: 1
293/420 on (-38301, -22514) (15426, 33207) (68542, 86355) hitlist: 1
292/420 off (-42704, -15950) (-24945, -8619) (-85074, -57059) hitlist: 1
291/420 on (-61239, -56325) (-28363, -20980) (-55652, -30421) hitlist: 1
290/420 off (-27003, -6291) (-90151, -56543) (-36412, -17679) hitlist: 1
289/420 of

160/420 on (-59969, -38119) (-32490, -11158) (-72565, -48356) hitlist: 1
159/420 on (14645, 18233) (70285, 80160) (-45717, -26265) hitlist: 1
158/420 on (39272, 60009) (21713, 53410) (29838, 55064) hitlist: 1
157/420 on (15487, 32009) (-17818, -4587) (-88133, -63626) hitlist: 1
156/420 on (-37353, -26663) (37926, 66858) (34514, 62276) hitlist: 1
155/420 on (-42549, -33653) (70071, 85597) (-16479, 7317) hitlist: 1
154/420 on (59743, 64863) (-24800, 7) (47840, 51569) hitlist: 1
153/420 on (32195, 41584) (-58378, -45157) (47203, 64230) hitlist: 1
152/420 on (-57862, -46129) (-39458, -13965) (52019, 57063) hitlist: 1
151/420 on (-66587, -39636) (2921, 14996) (-73004, -58075) hitlist: 1
150/420 on (-64068, -46550) (-49066, -35557) (-59104, -37575) hitlist: 1
149/420 on (46531, 63240) (-15497, 3408) (56333, 68319) hitlist: 1
148/420 on (17025, 38285) (-47077, -14251) (-77553, -54577) hitlist: 1
147/420 on (10776, 29554) (4929, 33616) (-82193, -56014) hitlist: 1
146/420 on (10985, 24603) (256

18/420 off (-22, -3) (6, 23) (-33, -23) hitlist: 9
17/420 on (-19, 34) (-5, 48) (-36, 8) hitlist: 22
16/420 off (-34, -20) (10, 27) (-39, -24) hitlist: 38
15/420 on (-22, 31) (-28, 17) (-46, -2) hitlist: 56
14/420 off (23, 39) (34, 43) (0, 18) hitlist: 71
13/420 on (-41, 4) (-33, 17) (-43, 10) hitlist: 85
12/420 off (10, 29) (2, 18) (-22, -10) hitlist: 105
11/420 on (-5, 47) (-41, 12) (-26, 27) hitlist: 106
10/420 off (-22, -6) (0, 11) (4, 18) hitlist: 125
9/420 on (-8, 36) (-48, 4) (-26, 19) hitlist: 129
8/420 on (-26, 25) (-26, 19) (-37, 17) hitlist: 145
7/420 on (-47, -3) (-8, 42) (-4, 49) hitlist: 162
6/420 on (-48, 2) (-38, 16) (-45, 6) hitlist: 173
5/420 on (-36, 18) (-7, 44) (1, 45) hitlist: 193
4/420 on (-32, 21) (-27, 18) (-43, 6) hitlist: 206
3/420 on (-24, 20) (-46, 8) (-10, 38) hitlist: 215
2/420 on (-44, 9) (-47, 7) (-18, 35) hitlist: 229
1/420 on (-35, 13) (-26, 26) (-47, -2) hitlist: 241
0/420 on (-8, 38) (-15, 37) (-49, 5) hitlist: 260
fn: i22.txt
part: 1
on: 650099
off

378/420 off (11750, 40798) (5403, 34501) (69392, 76971) hitlist: 1101
377/420 on (-74728, -50762) (49755, 73800) (-887, 19784) hitlist: 1153
376/420 off (-26694, -12337) (56020, 70984) (-50254, -30734) hitlist: 1179
375/420 off (26027, 32857) (58474, 78674) (36221, 62891) hitlist: 1220
374/420 off (-91328, -65484) (-7351, 22080) (11605, 46924) hitlist: 1242
373/420 off (-52932, -51838) (21389, 25746) (49616, 71107) hitlist: 1270
372/420 off (-15542, -4489) (61337, 76684) (-41765, -29786) hitlist: 1301
371/420 on (-31644, -6237) (7358, 28925) (73970, 84934) hitlist: 1333
370/420 off (39989, 66412) (-55818, -49092) (3921, 28857) hitlist: 1353
369/420 off (69115, 95898) (-25974, -2038) (-4973, 3980) hitlist: 1381
368/420 on (-3833, 3543) (-29112, -2048) (69842, 91959) hitlist: 1399
367/420 on (10847, 41288) (-91590, -58641) (15651, 35606) hitlist: 1429
366/420 off (76347, 91710) (-14687, 16646) (-25859, -5873) hitlist: 1454
365/420 off (37320, 53907) (-37056, -30571) (-68192, -42459) hitl

265/420 off (-25536, 3313) (27091, 50818) (-76013, -70562) hitlist: 4580
264/420 off (56007, 68228) (17457, 46813) (27499, 47379) hitlist: 4630
263/420 off (-8237, 24648) (-98122, -76129) (-16626, -11718) hitlist: 4651
262/420 off (-44344, -15201) (-24778, 6622) (-75251, -63023) hitlist: 4713
261/420 on (9529, 30152) (-62396, -39190) (-54970, -41011) hitlist: 4738
260/420 off (-69238, -33811) (-35451, -11427) (-68568, -56162) hitlist: 4790
259/420 off (12335, 28938) (55440, 86727) (22223, 31474) hitlist: 4826
258/420 off (46980, 55817) (-59630, -43170) (-39465, -15197) hitlist: 4851
257/420 on (-43172, -20564) (-15916, 3410) (55126, 75394) hitlist: 4882
256/420 on (35956, 60262) (49169, 87126) (-6572, 12137) hitlist: 4911
255/420 off (8685, 31733) (2760, 9002) (74103, 76071) hitlist: 4931
254/420 on (22913, 41733) (-69991, -52079) (-48649, -20477) hitlist: 5016
253/420 on (-64616, -44407) (-64266, -32556) (14623, 38546) hitlist: 5038
252/420 off (35485, 61206) (-24743, -300) (50470, 64

151/420 on (-66587, -39636) (2921, 14996) (-73004, -58075) hitlist: 7958
150/420 on (-64068, -46550) (-49066, -35557) (-59104, -37575) hitlist: 7977
149/420 on (46531, 63240) (-15497, 3408) (56333, 68319) hitlist: 7995
148/420 on (17025, 38285) (-47077, -14251) (-77553, -54577) hitlist: 8018
147/420 on (10776, 29554) (4929, 33616) (-82193, -56014) hitlist: 8051
146/420 on (10985, 24603) (25636, 55287) (-79921, -61711) hitlist: 8095
145/420 on (-13881, 12856) (36632, 69292) (56982, 70686) hitlist: 8108
144/420 on (-444, 13278) (22909, 49708) (58690, 72477) hitlist: 8127
143/420 on (-86133, -58057) (-27636, -4311) (-32792, -18848) hitlist: 8143
142/420 on (-57855, -35283) (-39073, -13640) (-80444, -56846) hitlist: 8160
141/420 on (-59280, -47097) (-39141, -17157) (33266, 47528) hitlist: 8177
140/420 on (15832, 37891) (-52792, -39704) (39855, 71748) hitlist: 8197
139/420 on (60965, 90946) (-14685, 10468) (20339, 49517) hitlist: 8242
138/420 on (23315, 42528) (-86791, -57800) (30250, 51456

36/420 on (-88874, -56782) (31756, 39638) (-6822, 22577) hitlist: 10834
35/420 on (43538, 74153) (-64825, -48158) (-2168, 17349) hitlist: 10900
34/420 on (17315, 32243) (-78736, -61172) (-48782, -36452) hitlist: 10923
33/420 on (51984, 58644) (-50863, -29318) (-62134, -38340) hitlist: 10935
32/420 on (-4302, 17673) (65726, 84939) (-24802, -1510) hitlist: 10975
31/420 on (-79023, -63744) (-29720, -25509) (-13690, 6232) hitlist: 10983
30/420 on (-16830, 10919) (-85429, -52622) (38655, 47942) hitlist: 10985
29/420 on (-60797, -42266) (-41852, -22898) (33729, 67815) hitlist: 10996
28/420 on (37522, 61641) (37254, 59332) (-25804, -10671) hitlist: 11007
27/420 on (-38651, -30945) (27847, 31103) (61105, 82258) hitlist: 11033
26/420 on (62170, 82174) (8028, 39169) (-12590, 14034) hitlist: 11059
25/420 on (40079, 61797) (-61859, -31418) (38360, 45565) hitlist: 11057
24/420 on (-9807, 20039) (-68478, -43761) (-57797, -44099) hitlist: 11098
23/420 on (3464, 34355) (-53573, -44765) (-72270, -45885